In [1]:
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss
import os

Using TensorFlow backend.


In [2]:
labels_path = os.getcwd() + "\\" + os.path.join( "data", "labels.csv", "")

In [3]:
labels_path

'C:\\Users\\Vladamir Mazin\\DeepNN\\data\\labels.csv\\'

In [4]:
data_labels = pd.read_csv(labels_path + "labels.csv")

In [5]:
type(data_labels)

pandas.core.frame.DataFrame

In [6]:
df = pd.DataFrame(np.random.randn(100, 2))
# df

In [7]:
# def get_train_test(data_labels):
#     msk = np.random.rand(len(data_labels)) < 0.8
#     train = data_labels[msk]
#     test = data_labels[~msk]
#     return train, test

In [8]:
# train, test = get_train_test(data_labels)

In [9]:
# train['id']

In [10]:
img_path = os.getcwd() + "\\" + os.path.join( "data", "train", "train", "")

In [11]:
# from itertools import islice
# for x in islice(train.iterrows(), 10):
#     print(x[1]['id'])

In [12]:
# # preparing the train dataset
# train_img = []
# for row in train.iterrows():
#     temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
#     temp_img = image.img_to_array(temp_img)
#     train_img.append(temp_img)
    
# #converting train images to array and applying mean subtraction processing
# train_img = np.array(train_img) 
# train_img = preprocess_input(train_img)

In [13]:
# # applying the same procedure with the test dataset
# test_img=[]
# for row in test.iterrows():
#     temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
#     temp_img = image.img_to_array(temp_img)
#     test_img.append(temp_img)

# test_img=np.array(test_img) 
# test_img=preprocess_input(test_img)

#### Let's consider all the data:

In [14]:
train = data_labels

In [16]:
# applying the same procedure with the test dataset
train_img = []
cnt = 1
for row in train.iterrows():
    temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
    temp_img = image.img_to_array(temp_img)
    train_img.append(temp_img)
    if cnt % 100 == 0:
        print("{} images processed".format(cnt))
    cnt += 1
train_img = np.array(train_img) 
train_img = preprocess_input(train_img)

100 images processed
200 images processed
300 images processed
400 images processed
500 images processed
600 images processed
700 images processed
800 images processed
900 images processed
1000 images processed
1100 images processed
1200 images processed
1300 images processed
1400 images processed
1500 images processed
1600 images processed
1700 images processed
1800 images processed
1900 images processed
2000 images processed
2100 images processed
2200 images processed
2300 images processed
2400 images processed
2500 images processed
2600 images processed
2700 images processed
2800 images processed
2900 images processed
3000 images processed
3100 images processed
3200 images processed
3300 images processed
3400 images processed
3500 images processed
3600 images processed
3700 images processed
3800 images processed
3900 images processed
4000 images processed
4100 images processed
4200 images processed
4300 images processed
4400 images processed
4500 images processed
4600 images process

In [17]:
# train_img.shape

In [18]:
from keras.models import Model

def vgg16_model(img_rows, img_cols, channel=1, num_classes=None, num_layers_to_keep=8):
    model = VGG16(weights='imagenet', include_top=True)

    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-1].outbound_nodes = []

    x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)

    #To set the first 8 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:num_layers_to_keep]:
        layer.trainable = False

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [20]:
train_y = np.asarray(train['breed'])

le = LabelEncoder()

train_y = le.fit_transform(train_y)

train_y=to_categorical(train_y)

train_y=np.array(train_y)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.2, random_state=42)

In [22]:
X_valid.shape

(2045, 224, 224, 3)

In [23]:
# Example to fine-tune on 3000 samples from Cifar10
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 120 
batch_size = 32
nb_epoch = 10

In [24]:
# Load our model
model_8_pretrained_layers = vgg16_model(img_rows, img_cols, channel, num_classes)
model_8_pretrained_layers.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [25]:
# Start Fine-tuning
model_8_pretrained_layers.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,
                              shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 156s 19ms/step - loss: 4.7769 - acc: 0.0362 - val_loss: 4.7680 - val_acc: 0.0592
Epoch 2/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.7573 - acc: 0.0945 - val_loss: 4.7545 - val_acc: 0.1100
Epoch 3/10
8177/8177 [==============================] - 147s 18ms/step - loss: 4.7428 - acc: 0.1411 - val_loss: 4.7426 - val_acc: 0.1364
Epoch 4/10
8177/8177 [==============================] - 146s 18ms/step - loss: 4.7292 - acc: 0.1805 - val_loss: 4.7322 - val_acc: 0.1672
Epoch 5/10
8177/8177 [==============================] - 147s 18ms/step - loss: 4.7152 - acc: 0.2003 - val_loss: 4.7199 - val_acc: 0.1814
Epoch 6/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.7008 - acc: 0.2265 - val_loss: 4.7091 - val_acc: 0.1980
Epoch 7/10
8177/8177 [==============================] - 146s 18ms/step - loss: 4.6859 - acc: 0.2512 - val_loss: 4.6947 - val_acc: 

In [26]:
# Start Fine-tuning
model_8_pretrained_layers.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,
                              shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 141s 17ms/step - loss: 4.6251 - acc: 0.3242 - val_loss: 4.6447 - val_acc: 0.2641
Epoch 2/10
8177/8177 [==============================] - 144s 18ms/step - loss: 4.6104 - acc: 0.3263 - val_loss: 4.6326 - val_acc: 0.2616
Epoch 3/10
8177/8177 [==============================] - 146s 18ms/step - loss: 4.5930 - acc: 0.3314 - val_loss: 4.6185 - val_acc: 0.2685
Epoch 4/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.5760 - acc: 0.3329 - val_loss: 4.6079 - val_acc: 0.2645
Epoch 5/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.5588 - acc: 0.3353 - val_loss: 4.5923 - val_acc: 0.2655
Epoch 6/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.5418 - acc: 0.3374 - val_loss: 4.5737 - val_acc: 0.2782
Epoch 7/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.5254 - acc: 0.3388 - val_loss: 4.5640 - val_acc: 

In [27]:
# Start Fine-tuning
model_8_pretrained_layers.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,
                              shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 147s 18ms/step - loss: 4.4542 - acc: 0.3503 - val_loss: 4.5154 - val_acc: 0.2631
Epoch 2/10
8177/8177 [==============================] - 146s 18ms/step - loss: 4.4375 - acc: 0.3532 - val_loss: 4.4965 - val_acc: 0.2650
Epoch 3/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.4217 - acc: 0.3565 - val_loss: 4.4884 - val_acc: 0.2748
Epoch 4/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.4036 - acc: 0.3606 - val_loss: 4.4662 - val_acc: 0.2836
Epoch 5/10
8177/8177 [==============================] - 144s 18ms/step - loss: 4.3843 - acc: 0.3619 - val_loss: 4.4551 - val_acc: 0.2719
Epoch 6/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.3674 - acc: 0.3636 - val_loss: 4.4380 - val_acc: 0.2846
Epoch 7/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.3486 - acc: 0.3664 - val_loss: 4.4367 - val_acc: 

In [28]:
# Start Fine-tuning
model_8_pretrained_layers.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,
                              shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 143s 18ms/step - loss: 4.2775 - acc: 0.3684 - val_loss: 4.3830 - val_acc: 0.2773
Epoch 2/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.2680 - acc: 0.3657 - val_loss: 4.3592 - val_acc: 0.2817
Epoch 3/10
8177/8177 [==============================] - 144s 18ms/step - loss: 4.2449 - acc: 0.3690 - val_loss: 4.3454 - val_acc: 0.2773
Epoch 4/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.2277 - acc: 0.3702 - val_loss: 4.3324 - val_acc: 0.2866
Epoch 5/10
8177/8177 [==============================] - 144s 18ms/step - loss: 4.2094 - acc: 0.3709 - val_loss: 4.3175 - val_acc: 0.2797
Epoch 6/10
8177/8177 [==============================] - 144s 18ms/step - loss: 4.1926 - acc: 0.3699 - val_loss: 4.3117 - val_acc: 0.2782
Epoch 7/10
8177/8177 [==============================] - 144s 18ms/step - loss: 4.1735 - acc: 0.3709 - val_loss: 4.2964 - val_acc: 

In [ ]:
# Start Fine-tuning
model_8_pretrained_layers.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,
                              shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 150s 18ms/step - loss: 4.1012 - acc: 0.3754 - val_loss: 4.2335 - val_acc: 0.2875
Epoch 2/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.0823 - acc: 0.3762 - val_loss: 4.2322 - val_acc: 0.2782
Epoch 3/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.0647 - acc: 0.3769 - val_loss: 4.2067 - val_acc: 0.2822
Epoch 4/10
8177/8177 [==============================] - 146s 18ms/step - loss: 4.0512 - acc: 0.3761 - val_loss: 4.1993 - val_acc: 0.2836
Epoch 5/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.0322 - acc: 0.3764 - val_loss: 4.1829 - val_acc: 0.2797
Epoch 6/10
8177/8177 [==============================] - 145s 18ms/step - loss: 4.0158 - acc: 0.3756 - val_loss: 4.1694 - val_acc: 0.2841
Epoch 7/10
8177/8177 [==============================] - 145s 18ms/step - loss: 3.9982 - acc: 0.3767 - val_loss: 4.1702 - val_acc: 

In [50]:
# Start Fine-tuning
# model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

In [47]:
from vis.utils import utils
from keras import activations

In [48]:
# copy_for_vis_model = keras.models.clone_model(model_8_pretrained_layers)

In [49]:
# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model_8_pretrained_layers, 'predictions')

In [50]:
layer_idx

22

In [51]:
# Swap softmax with linear
model_8_pretrained_layers.layers[layer_idx].activation = activations.linear
model_8_pretrained_layers = utils.apply_modifications(model_8_pretrained_layers)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/tmp/vldbiqb5.h5'

### Let's try model with [2 more conv layers] that is leaving 8 + 3 + 4 = 15 layers

In [24]:
# Load our model
model_15_pretrained_layers = vgg16_model(img_rows, img_cols, channel, num_classes, num_layers_to_keep=15)
model_15_pretrained_layers.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
# Start Fine-tuning
model_15_pretrained_layers.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,
                               shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 112s 14ms/step - loss: 4.7829 - acc: 0.0212 - val_loss: 4.7729 - val_acc: 0.0489
Epoch 2/10
8177/8177 [==============================] - 102s 12ms/step - loss: 4.7640 - acc: 0.0805 - val_loss: 4.7585 - val_acc: 0.1086
Epoch 3/10
8177/8177 [==============================] - 101s 12ms/step - loss: 4.7492 - acc: 0.1479 - val_loss: 4.7466 - val_acc: 0.1692
Epoch 4/10
8177/8177 [==============================] - 101s 12ms/step - loss: 4.7348 - acc: 0.1924 - val_loss: 4.7352 - val_acc: 0.2083
Epoch 5/10
8177/8177 [==============================] - 100s 12ms/step - loss: 4.7211 - acc: 0.2555 - val_loss: 4.7226 - val_acc: 0.2543
Epoch 6/10
8177/8177 [==============================] - 101s 12ms/step - loss: 4.7068 - acc: 0.2819 - val_loss: 4.7105 - val_acc: 0.2733
Epoch 7/10
8177/8177 [==============================] - 111s 14ms/step - loss: 4.6924 - acc: 0.3050 - val_loss: 4.6981 - val_acc: 

In [43]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

2045/2045 [==============================] - 17s 8ms/step


In [44]:
# Cross-entropy loss score
score = log_loss(Y_valid, predictions_valid)

In [47]:
score

4.7878120063278082